In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig

In [ ]:
# LOGIN USING THE PREVIOUS TOKEN
base_model_id = "bigscience/mt0-xl"
#base_model_id = "bigscience/mt0-xl"
#lora_model_id = "iproskurina/s-nlp-mt0-xl-detox-orpo-detoxify-prompt"
lora_model_id = "Dalfaxy/mt0_xl_french_detox_v3"

#from huggingface_hub import login

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_id, device_map="auto")


model = PeftModel.from_pretrained(base_model, lora_model_id)
def detoxify(text):
  prompt = "détoxifier: "+ text
  encodings = tokenizer(prompt, return_tensors='pt').to(model.device)

  outputs = model.generate(
  **encodings,
  max_length=128,
  num_beams=10,
  no_repeat_ngram_size=3,
  repetition_penalty=1.2,
  num_beam_groups=5,
  diversity_penalty=2.5,
  num_return_sequences=5,
  early_stopping=True,
  )

  sequences = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  return sequences

toxic_text = "Tu es un idiot"
result = detoxify(toxic_text)

print(result)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

Group Beam Search is scheduled to be moved to a `custom_generate` repository in v4.55.0. To prevent loss of backward compatibility, add `trust_remote_code=True` to your `generate` call.


["Vous n'êtes pas intelligent", "Vous n'êtes pas bon", "tu n'es pas intelligent", "Tu n'es pas intelligent", "Votre cerveau n'est pas bon"]


In [ ]:
import torch

In [ ]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer

sta = pipeline("text-classification", model="textdetox/xlmr-large-toxicity-classifier")
sim = SentenceTransformer('sentence-transformers/LaBSE')

Device set to use cuda:0


In [ ]:
def select_best_output(text, detoxs, sta, sim):
    vals = []
    for detox in detoxs:
        emb = sim.encode([text, detox], convert_to_tensor=True)

        sim_val = (emb[0] * emb[1]).sum()
        sta_val = sta(detox)[0]
        sta_score = sta_val['score']
        if sta_val['label'] == 'LABEL_1':
            sta_score = 1 - sta_score

        vals.append((detox, (sim_val*sta_score).item()))
    detox, _ = max(vals, key=lambda x: x[1])
    return detox

In [ ]:
def build_prompt(text):
    prefix = "Détoxifier: "
    return f"{prefix}{text}"

def detoxify_dataframe(df, text_col="toxic_sentence", lang_col="lang", batch_size=8):
    prompts = []
    original_indices = []

    # Construction des prompts et indices
    for idx, row in df.iterrows():
        prompt = build_prompt(row[text_col])
        prompts.append(prompt)
        original_indices.append(idx)

    results = []

    # Traitement en batch
    for i in tqdm(range(0, len(prompts), batch_size), desc="Detoxifying"):
        batch_prompts = prompts[i:i + batch_size]
        encodings = tokenizer(batch_prompts, return_tensors='pt', padding=True, truncation=True).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **encodings,
                max_length=128,
                num_beams=10,
                no_repeat_ngram_size=3,
                repetition_penalty=1.2,
                num_beam_groups=5,
                diversity_penalty=2.5,
                num_return_sequences=1,
                early_stopping=True,
            )

        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        results.extend(decoded)

    # Création d'une nouvelle colonne avec les résultats alignés
    df_result = df.copy()
    df_result["neutral_sentence"] = pd.Series(results, index=original_indices)

    return df_result


In [ ]:
# prompt: charge le dataframe et applique la fonction detoxify sur la colonne toxic_sentence
import pandas as pd
from tqdm import tqdm
df = pd.read_csv("test_inputs_upd.tsv",sep="\t")

In [ ]:
detoxified_df = detoxify_dataframe(df[df["lang"]=="fr"],batch_size=16)

Detoxifying: 100%|██████████| 38/38 [05:33<00:00,  8.77s/it]


In [ ]:
detoxified_df.head()

,toxic_sentence,neutral_sentence,lang
6000,# Qu'est ce que tu viens de putain de dire à p...,Qu'est-ce que tu viens de dire à propos de moi?,fr
6001,Qu'est ce que tu viens de putain de dire à pro...,Qu'est-ce que tu viens de dire à propos des mi...,fr
6002,", je t'offre une vraie traduction maison : Tu...",", je t'offre une vraie traduction maison : Tu ...",fr
6003,"Oh non, je ne vais pas bouger, et c'est juste...","Oh non, je ne vais pas bouger, et c'est juste ...",fr
6004,"Faut pas downvote ce genre de truc, c'est cont...","Vous ne devriez pas downvote ce genre de truc,...",fr


In [ ]:
df["neutral_sentence"] = df["toxic_sentence"]
df_copy = df.copy()
df_copy.loc[df_copy["lang"] == "fr", "neutral_sentence"] = detoxified_df["neutral_sentence"]
df = df_copy
df.head()

,toxic_sentence,neutral_sentence,lang
0,Просто забирайся нахуй з моєї голови.,Просто забирайся нахуй з моєї голови.,uk
1,"Тримай сраний навігатор, тварина, а то засунь ...","Тримай сраний навігатор, тварина, а то засунь ...",uk
2,"мать тваю,ану іді сюда, гавно собачие,думал ко...","мать тваю,ану іді сюда, гавно собачие,думал ко...",uk
3,І всі на один час блять(,І всі на один час блять(,uk
4,"та бо сука, молоко хоче гречку, а гречка уйоба...","та бо сука, молоко хоче гречку, а гречка уйоба...",uk


In [ ]:
df[df["lang"]=="fr"]

,toxic_sentence,neutral_sentence,lang
6000,# Qu'est ce que tu viens de putain de dire à p...,# Qu'est-ce que tu viens de dire à propos de moi?,fr
6001,Qu'est ce que tu viens de putain de dire à pro...,Qu'est-ce que tu viens de dire à propos des mi...,fr
6002,", je t'offre une vraie traduction maison : Tu...",", je t'offre une vraie traduction maison : Tu ...",fr
6003,"Oh non, je ne vais pas bouger, et c'est juste...","Oh non, je ne vais pas bouger, et c'est juste ...",fr
6004,"Faut pas downvote ce genre de truc, c'est cont...","Vous ne devez pas downvote ce genre de truc, c...",fr
...,...,...,...
6595,tu te fais chier dans la vie ??,Vous vous trompez dans la vie ??,fr
6596,Ico est un gros trou du cul qui passe son temp...,Ico passe son temps à effacer toutes les sourc...,fr
6597,Je n ai jamais rencontré d individu aussi bas ...,Je n ai jamais rencontré d'individu aussi bas ...,fr
6598,Au lieu de faire chier les gens sur les sites ...,Au lieu de gâcher les gens sur les sites Inter...,fr


In [ ]:
df.to_csv("french_sub_mt0_xl_v3_filtered.tsv",sep="\t",index=False)